In [1]:
import json
import polars as pl
import numpy as np

In [2]:
df_vorgang = pl.read_json('vorgang.json')
df_gesetze = df_vorgang.filter(pl.col('vorgangstyp') == 'Gesetzgebung')
pl.Config(fmt_str_lengths=100)
df_gesetze['beratungsstand'].unique()

beratungsstand
str
"""Zusammengeführt mit... (siehe Vorgangsablauf)"""
"""Noch nicht beraten"""
"""Verabschiedet"""
"""Den Ausschüssen zugewiesen"""
"""Für erledigt erklärt"""
"""Zurückgezogen"""
"""Angenommen"""
"""Dem Bundestag zugeleitet - Noch nicht beraten"""
"""Einbringung abgelehnt"""


In [3]:
df_vorgangspos = pl.read_json('vorgangsposition.json')
df_vorgangspos = df_vorgangspos.with_columns(
   pl.col("datum").str.to_date(format="%Y-%m-%d")
)
df_vorgangspos = df_vorgangspos.filter((pl.col('vorgangstyp') == 'Gesetzgebung') & (pl.col('typ') == 'Vorgangsposition'))
df_vorgang_duration = df_vorgangspos.group_by(['vorgang_id']).agg([pl.col('datum').min().alias('min'), pl.col('datum').max().alias('max')])
df_vorgang_duration = df_vorgang_duration.with_columns((pl.col('max') - pl.col('min')).dt.days().alias('vorgangsdauer'))

In [4]:
df_gesetze = df_gesetze.join(df_vorgang_duration.select(['vorgang_id', 'vorgangsdauer']), left_on='id', right_on='vorgang_id', how='left')

In [5]:
df_gesetze.write_json('gesetze.json', row_oriented=True)

In [6]:
df_gesetze_public = df_gesetze.filter(pl.col('beratungsstand') == 'Verkündet')
bins = [7, 14, 30, 60, 120, 240, 365]
duration_bin = np.digitize(df_gesetze_public.select('vorgangsdauer').to_series(), bins, right=True)
df_gesetze_public.select('vorgangsdauer').to_series().hist(bins)

break_point,category,vorgangsdauer_count
f64,cat,u32
7.0,"""(-inf, 7.0]""",2
14.0,"""(7.0, 14.0]""",8
30.0,"""(14.0, 30.0]""",15
60.0,"""(30.0, 60.0]""",17
120.0,"""(60.0, 120.0]""",64
240.0,"""(120.0, 240.0]""",48
365.0,"""(240.0, 365.0]""",5
inf,"""(365.0, inf]""",3


In [7]:
df_gesetze_public_bins = df_gesetze_public.with_columns(pl.from_numpy(duration_bin, schema=['duration_bin']))
df_gesetze_public_bins.filter(pl.col('duration_bin') == pl.col('duration_bin').min())

id,abstract,beratungsstand,vorgangstyp,gesta,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,aktualisiert,deskriptor,titel,datum,vorgang_verlinkung,ratsdok,kom,vorgangsdauer,duration_bin
str,str,str,str,str,list[str],str,i64,list[str],list[str],str,list[struct[3]],str,str,list[struct[5]],str,str,i64,i64
"""289160""","""Zustimmung zu den am 5. Juli 2022 in Brüssel unterzeichneten Protokollen zum Nordatlantikvertrag üb…","""Verkündet""","""Gesetzgebung""","""XC002""","[""Verteidigung""]","""Vorgang""",20,"[""Nein, laut Gesetzentwurf (Drs 20/2534)"", ""Nein, laut Verkündung (BGBl I)""]","[""Fraktion BÜNDNIS 90/DIE GRÜNEN"", ""Fraktion der CDU/CSU"", … ""Fraktion der SPD""]","""2023-04-27T11:12:50+02:00""","[{true,""Finnland"",""Geograph. Begriffe""}, {false,""Militärbündnis"",""Sachbegriffe""}, … {true,""Schweden"",""Geograph. Begriffe""}]","""Gesetz zu den Protokollen zum Nordatlantikvertrag über den Beitritt der Republik Finnland und des K…","""2022-07-08""",null,null,null,7,0
"""283063""","""Impfpflicht gegen COVID-19 bzw. Nachweis über Genesung oder Kontraindikation für Beschäftigte besti…","""Verkündet""","""Gesetzgebung""","""M002""","[""Gesundheit""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 20/188)"", ""Ja, laut Verkündung (BGBl I)""]","[""Fraktion BÜNDNIS 90/DIE GRÜNEN"", ""Fraktion der FDP"", ""Fraktion der SPD""]","""2022-07-26T19:57:25+02:00""","[{false,""Apotheker"",""Sachbegriffe""}, {false,""Arbeitnehmer-Entsendegesetz"",""Rechtsmaterialien""}, … {false,""Zahnarzt"",""Sachbegriffe""}]","""Gesetz zur Stärkung der Impfprävention gegen COVID-19 und zur Änderung weiterer Vorschriften im Zus…","""2021-12-10""","[{""Entschließungsantrag der Fraktion der AfD"",""Entschließungsantrag zum Gesetz zur Stärkung der Impfprävention gegen COVID-19 und zur Änderung weiterer Vorschriften im Zusammenhang mit der COVID-19-Pandemie"",""283133"",20,null}]",null,null,4,0


In [8]:
df_gesetze_public_bins.filter(pl.col('duration_bin') == pl.col('duration_bin').max())

id,abstract,beratungsstand,vorgangstyp,gesta,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,aktualisiert,deskriptor,titel,datum,vorgang_verlinkung,ratsdok,kom,vorgangsdauer,duration_bin
str,str,str,str,str,list[str],str,i64,list[str],list[str],str,list[struct[3]],str,str,list[struct[5]],str,str,i64,i64
"""290305""","""Anpassung an Vorschriften des EU-Rechts durch Erweiterung der Agrarstrukturerhebung um zusätzlich v…","""Verkündet""","""Gesetzgebung""","""F007""","[""Landwirtschaft und Ernährung""]","""Vorgang""",20,"[""Nein, laut Gesetzentwurf (Drs 362/22)"", ""Nein, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","""2023-09-06T14:57:55+02:00""","[{true,""Agrarstatistik"",""Sachbegriffe""}, {true,""Agrarstatistikgesetz"",""Rechtsmaterialien""}, … {false,""Verordnung der EU"",""Sachbegriffe""}]","""Gesetz zur Änderung des Agrarstatistikgesetzes und des Lebensmittelspezialitätengesetzes""","""2023-08-31""",null,null,null,391,7
"""284432""","""Entlastung einkommensschwächerer Haushalte durch Einführung eines einmaligen nach der Anzahl der be…","""Verkündet""","""Gesetzgebung""","""P003""","[""Gesellschaftspolitik, soziale Gruppen"", ""Raumordnung, Bau- und Wohnungswesen"", ""Soziale Sicherung""]","""Vorgang""",20,"[""Nein, laut Gesetzentwurf (Drs 20/689)"", ""Nein, laut Verkündung (BGBl I)""]","[""Fraktion BÜNDNIS 90/DIE GRÜNEN"", ""Fraktion der FDP"", ""Fraktion der SPD""]","""2023-08-21T14:27:18+02:00""","[{false,""Aufstiegsfortbildung"",""Sachbegriffe""}, {false,""Ausbildungsförderung"",""Sachbegriffe""}, … {false,""Wohngeld"",""Sachbegriffe""}]","""Gesetz zur Gewährung eines einmaligen Heizkostenzuschusses aufgrund stark gestiegener Energiekosten…","""2023-08-14""","[{""Entschließungsantrag der Fraktion der CDU/CSU"",""Entschließungsantrag zum Gesetz zur Gewährung eines einmaligen Heizkostenzuschusses aufgrund stark gestiegener Energiekosten (Heizkostenzuschussgesetz - HeizkZuschG)"",""285421"",20,null}, {""Entschließungsantrag der Fraktion DIE LINKE"",""Entschließungsantrag zum Gesetz zur Gewährung eines einmaligen Heizkostenzuschusses aufgrund stark gestiegener Energiekosten (Heizkostenzuschussgesetz - HeizkZuschG)"",""285431"",20,null}]",null,null,545,7
"""287570""","""Weitere Erhöhung der Regionalisierungsmittel im Jahr 2022 um insgesamt 3,7 Mrd Euro zur finanzielle…","""Verkündet""","""Gesetzgebung""","""J003""","[""Verkehr"", ""Öffentliche Finanzen, Steuern und Abgaben""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 20/1739)"", ""Ja, laut Verkündung (BGBl I)""]","[""Fraktion BÜNDNIS 90/DIE GRÜNEN"", ""Fraktion der FDP"", ""Fraktion der SPD""]","""2023-08-02T14:41:22+02:00""","[{false,""Bericht der Bundesregierung"",""Sachbegriffe""}, {false,""Bundesmittel"",""Sachbegriffe""}, … {true,""Öffentlicher Personennahverkehr"",""Sachbegriffe""}]","""Siebtes Gesetz zur Änderung des Regionalisierungsgesetzes""","""2023-07-26""","[{""Entschließungsantrag der Fraktion der CDU/CSU"",""Entschließungsantrag zum Siebten Gesetz zur Änderung des Regionalisierungsgesetzes"",""287866"",20,null}, {""Entschließungsantrag der Fraktion der AfD"",""Entschließungsantrag zum Siebten Gesetz zur Änderung des Regionalisierungsgesetzes"",""287876"",20,null}]",null,null,442,7
